# Energy conversion integration

Once the heat recovery is implemented, the energy balance of the process system is closed by converting energy resources like natural and electricity into useful heat, cold or refrigeration heat loads for the process.

## Grand Composite Curve


In [ ]:
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Function to create the Grand Composite Curve plot
def create_gcc_plot(Q_gcc, T_hc, title):
    fig = go.Figure()

    # Add the Grand Composite Curve without legend
    fig.add_trace(go.Scatter(x=Q_gcc, y=T_hc, mode='lines', name=None))

    # Add horizontal and vertical reference lines
    fig.add_shape(go.layout.Shape(type='line', x0=0, x1=0, y0=min(T_hc), y1=max(T_hc), line=dict(dash='dash')))
    fig.add_shape(go.layout.Shape(type='line', x0=min(Q_gcc), x1=max(Q_gcc), y0=0, y1=0, line=dict(dash='dash')))

    # Set axis labels and title
    fig.update_layout(
        xaxis_title='Heat Load (kW)',
        yaxis_title='Temperature (°C)',
        title=dict(text=title, x=0.5),  # Center the title
        paper_bgcolor='white',  # Set the background color
    )

    return fig

# Read JSON for heat integration data for each case
file_names = [
    'codes_02_heat_recovery/result/brewery-57.json',
    'codes_02_heat_recovery/result/brewery-59NoRivellaNoDigester.json',
    'codes_02_heat_recovery/result/brewery-59_no_digester.json',
    'codes_02_heat_recovery/result/brewery-59NoRivella.json'
]

titles = [
    'With Rivella and Digester',
    'Without Rivella and Digester',
    'Only with Rivella',
    'Only with Digester'
]

# Create subplots with a 2x2 grid
fig = make_subplots(rows=2, cols=2, subplot_titles=titles)

# Loop through each case and add the corresponding subplot
for i, file_name in enumerate(file_names, start=1):
    integ = json.loads(open(file_name).read())
    hotcc = integ["results"]["graph"][0][0][1]["data"][0]["curve"]
    coldcc = integ["results"]["graph"][0][0][1]["data"][1]["curve"]
    T_hc = [(i["T"] - 273.15) for i in hotcc]
    Q_hc = [i["Q"] for i in hotcc]
    T_cc = [(i["T"] - 273.15) for i in coldcc]
    Q_cc = [i["Q"] for i in coldcc]

    deltaT_min = 5
    T_hc_ori = [(i + deltaT_min/2) for i in T_hc]
    Q_gcc = np.zeros(len(Q_cc))

    for j in range(len(Q_cc)):
        Q_gcc[j] = Q_cc[j] - Q_hc[j]

    row = (i - 1) // 2 + 1
    col = (i - 1) % 2 + 1

    fig.add_trace(go.Scatter(x=Q_gcc, y=T_hc, mode='lines', name=None), row=row, col=col)

    # Add horizontal and vertical reference lines to each subplot
    fig.add_shape(go.layout.Shape(type='line', x0=0, x1=0, y0=min(T_hc), y1=max(T_hc), line=dict(dash='dash')), row=row, col=col)
    fig.add_shape(go.layout.Shape(type='line', x0=min(Q_gcc), x1=max(Q_gcc), y0=0, y1=0, line=dict(dash='dash')), row=row, col=col)

    # Set axis labels and title for the subplot
    fig.update_xaxes(title_text='Heat Load (kW)', row=row, col=col)
    fig.update_yaxes(title_text='Temperature (°C)', row=row, col=col)
    fig.update_layout(title=dict(text='Grand Composite Curve', x=0.5), paper_bgcolor='white', showlegend = False)

# Show the plot
fig.show()

In [ ]:
import matplotlib.pyplot as plt
import json
import numpy as np
import pandas as pd
from IPython.display import display, HTML

plt.switch_backend('agg')

# read json for heat integration data
file_name = open('codes_02_heat_recovery/result/brewery-57.json')
integ = json.loads(file_name.read())

# we used .keys() and json-reader to locate the data of interest (hot and cold composite curves)
hotcc = integ["results"]["graph"][0][0][1]["data"][0]["curve"]
coldcc = integ["results"]["graph"][0][0][1]["data"][1]["curve"]

# extract the temperature (T in Kelvin) and heat (Q in kW)
T_hc = [i["T"] for i in hotcc]
Q_hc = [i["Q"] for i in hotcc]
T_cc = [i["T"] for i in coldcc]
Q_cc = [i["Q"] for i in coldcc]

# compute the shifted carnot factor (1 - T0/T) and Treference = 298.15K
Tref = 298.15 
ca_hc = [(1-Tref/i) for i in T_hc]
ca_cc = [(1-Tref/i) for i in T_cc]

# compute back the original carnot factors given a deltaT_min
#deltaT_min = 40
deltaT_min = 5
T_hc_ori = [(i+deltaT_min/2) for i in T_hc]
T_cc_ori = [(i-deltaT_min/2) for i in T_cc]
ca_hc_ori = [(1-298.15/i) for i in T_hc_ori]
ca_cc_ori = [(1-298.15/i) for i in T_cc_ori]

# prepare the second y-axis for plotting
# the minim carnot value is extracted from the original cooling. Round it to the lowest .0
# the maximum carnot carnot value is extracted from the original heating. Round it to the highest .0
ca_min = round(min(ca_cc_ori) - 0.05, 1)
ca_max = round(max(ca_hc_ori) + 0.05, 1)
# generate an array with the carnot list from min to max
ca_values = np.arange(ca_min, ca_max, 0.2)
# compute the corresponding temperature values in celcius
Tref_ce = 298.15
T_ce = [int(Tref_ce/(1-i) - 273.15) for i in ca_values]


T_cc = [(i-273.15) for i in T_cc]
T_cc_ori = [(i-273.15) for i in T_cc_ori]
T_hc = [(i-273.15) for i in T_hc]
T_hc_ori = [(i-273.15) for i in T_hc_ori]


Q_gcc = np.zeros(len(Q_cc))

for i in range(len(Q_cc)):
  Q_gcc[i] = Q_cc[i] - Q_hc[i]  

Heat_inter = []
intersections = []
for i in range(len(Q_gcc)):
  if (Q_gcc[i]<=10):
    if (T_hc[i]>=-30 and T_hc[i]<=200):
      intersections.append(T_hc[i])
      Heat_inter.append(Q_gcc[i])


upper_bound = 0
lower_bound = 30000
for i in range(len(Q_hc)):
  if (ca_hc_ori[i] < (np.max(ca_hc_ori)) and Q_hc[i] > upper_bound):
    upper_bound = Q_hc[i]
  if (ca_cc_ori[i] > np.min(ca_cc_ori) and Q_cc[i] < lower_bound):
    lower_bound = Q_cc[i]




cold_utilities = lower_bound
total_w = 0

for i in range(len(Q_cc)):
  if ((ca_cc_ori[i] > 0.2) and (Q_cc[i] > total_w)):
    total_w = Q_cc[i]

hot_utilities = total_w - upper_bound


data_heat_recovery = {'With Rivella and Digester': ['Cold Utilities', 'Hot Utilities', 'Minimum Energy Requirements', 'Maximum Heat Recovery Potential'],
        'Heat (kW)': [cold_utilities, hot_utilities, cold_utilities+hot_utilities, np.round(upper_bound - lower_bound, 2)]}

dfs = pd.DataFrame(data_heat_recovery)
dfs = dfs.set_index('With Rivella and Digester')
dfs = dfs.round(2)
HTML(dfs.to_html())

In [ ]:
import matplotlib.pyplot as plt
import json
import numpy as np
import pandas as pd
from IPython.display import display, HTML

plt.switch_backend('agg')

# read json for heat integration data
file_name = open('codes_02_heat_recovery/result/brewery-59NoRivellaNoDigester.json')
integ = json.loads(file_name.read())

# we used .keys() and json-reader to locate the data of interest (hot and cold composite curves)
hotcc = integ["results"]["graph"][0][0][1]["data"][0]["curve"]
coldcc = integ["results"]["graph"][0][0][1]["data"][1]["curve"]

# extract the temperature (T in Kelvin) and heat (Q in kW)
T_hc = [i["T"] for i in hotcc]
Q_hc = [i["Q"] for i in hotcc]
T_cc = [i["T"] for i in coldcc]
Q_cc = [i["Q"] for i in coldcc]

# compute the shifted carnot factor (1 - T0/T) and Treference = 298.15K
Tref = 298.15 
ca_hc = [(1-Tref/i) for i in T_hc]
ca_cc = [(1-Tref/i) for i in T_cc]

# compute back the original carnot factors given a deltaT_min
#deltaT_min = 40
deltaT_min = 5
T_hc_ori = [(i+deltaT_min/2) for i in T_hc]
T_cc_ori = [(i-deltaT_min/2) for i in T_cc]
ca_hc_ori = [(1-298.15/i) for i in T_hc_ori]
ca_cc_ori = [(1-298.15/i) for i in T_cc_ori]

# prepare the second y-axis for plotting
# the minim carnot value is extracted from the original cooling. Round it to the lowest .0
# the maximum carnot carnot value is extracted from the original heating. Round it to the highest .0
ca_min = round(min(ca_cc_ori) - 0.05, 1)
ca_max = round(max(ca_hc_ori) + 0.05, 1)
# generate an array with the carnot list from min to max
ca_values = np.arange(ca_min, ca_max, 0.2)
# compute the corresponding temperature values in celcius
Tref_ce = 298.15
T_ce = [int(Tref_ce/(1-i) - 273.15) for i in ca_values]


T_cc = [(i-273.15) for i in T_cc]
T_cc_ori = [(i-273.15) for i in T_cc_ori]
T_hc = [(i-273.15) for i in T_hc]
T_hc_ori = [(i-273.15) for i in T_hc_ori]


Q_gcc = np.zeros(len(Q_cc))

for i in range(len(Q_cc)):
  Q_gcc[i] = Q_cc[i] - Q_hc[i]  

Heat_inter = []
intersections = []
for i in range(len(Q_gcc)):
  if (Q_gcc[i]<=10):
    if (T_hc[i]>=-30 and T_hc[i]<=200):
      intersections.append(T_hc[i])
      Heat_inter.append(Q_gcc[i])


upper_bound = 0
lower_bound = 30000
for i in range(len(Q_hc)):
  if (ca_hc_ori[i] < (np.max(ca_hc_ori)) and Q_hc[i] > upper_bound):
    upper_bound = Q_hc[i]
  if (ca_cc_ori[i] > np.min(ca_cc_ori) and Q_cc[i] < lower_bound):
    lower_bound = Q_cc[i]




cold_utilities = lower_bound
total_w = 0

for i in range(len(Q_cc)):
  if ((ca_cc_ori[i] > 0.1) and (Q_cc[i] > total_w)):
    total_w = Q_cc[i]

hot_utilities = total_w - upper_bound


data_heat_recovery = {'Without Rivella and Digester': ['Cold Utilities', 'Hot Utilities', 'Minimum Energy Requirements', 'Maximum Heat Recovery Potential'],
        'Heat (kW)': [cold_utilities, hot_utilities, cold_utilities+hot_utilities, np.round(upper_bound - lower_bound, 2)]}

dfs = pd.DataFrame(data_heat_recovery)
dfs = dfs.set_index('Without Rivella and Digester')
dfs = dfs.round(2)
HTML(dfs.to_html())

In [ ]:
import matplotlib.pyplot as plt
import json
import numpy as np
import pandas as pd
from IPython.display import display, HTML

plt.switch_backend('agg')

# read json for heat integration data
file_name = open('codes_02_heat_recovery/result/brewery-59_no_digester.json')
integ = json.loads(file_name.read())

# we used .keys() and json-reader to locate the data of interest (hot and cold composite curves)
hotcc = integ["results"]["graph"][0][0][1]["data"][0]["curve"]
coldcc = integ["results"]["graph"][0][0][1]["data"][1]["curve"]

# extract the temperature (T in Kelvin) and heat (Q in kW)
T_hc = [i["T"] for i in hotcc]
Q_hc = [i["Q"] for i in hotcc]
T_cc = [i["T"] for i in coldcc]
Q_cc = [i["Q"] for i in coldcc]

# compute the shifted carnot factor (1 - T0/T) and Treference = 298.15K
Tref = 298.15 
ca_hc = [(1-Tref/i) for i in T_hc]
ca_cc = [(1-Tref/i) for i in T_cc]

# compute back the original carnot factors given a deltaT_min
#deltaT_min = 40
deltaT_min = 5
T_hc_ori = [(i+deltaT_min/2) for i in T_hc]
T_cc_ori = [(i-deltaT_min/2) for i in T_cc]
ca_hc_ori = [(1-298.15/i) for i in T_hc_ori]
ca_cc_ori = [(1-298.15/i) for i in T_cc_ori]

# prepare the second y-axis for plotting
# the minim carnot value is extracted from the original cooling. Round it to the lowest .0
# the maximum carnot carnot value is extracted from the original heating. Round it to the highest .0
ca_min = round(min(ca_cc_ori) - 0.05, 1)
ca_max = round(max(ca_hc_ori) + 0.05, 1)
# generate an array with the carnot list from min to max
ca_values = np.arange(ca_min, ca_max, 0.2)
# compute the corresponding temperature values in celcius
Tref_ce = 298.15
T_ce = [int(Tref_ce/(1-i) - 273.15) for i in ca_values]


T_cc = [(i-273.15) for i in T_cc]
T_cc_ori = [(i-273.15) for i in T_cc_ori]
T_hc = [(i-273.15) for i in T_hc]
T_hc_ori = [(i-273.15) for i in T_hc_ori]


Q_gcc = np.zeros(len(Q_cc))

for i in range(len(Q_cc)):
  Q_gcc[i] = Q_cc[i] - Q_hc[i]  

Heat_inter = []
intersections = []
for i in range(len(Q_gcc)):
  if (Q_gcc[i]<=10):
    if (T_hc[i]>=-30 and T_hc[i]<=200):
      intersections.append(T_hc[i])
      Heat_inter.append(Q_gcc[i])


upper_bound = 0
lower_bound = 30000
for i in range(len(Q_hc)):
  if (ca_hc_ori[i] < (np.max(ca_hc_ori)) and Q_hc[i] > upper_bound):
    upper_bound = Q_hc[i]
  if (ca_cc_ori[i] > np.min(ca_cc_ori) and Q_cc[i] < lower_bound):
    lower_bound = Q_cc[i]




cold_utilities = lower_bound
total_w = 0

for i in range(len(Q_cc)):
  if ((ca_cc_ori[i] > 0.2) and (Q_cc[i] > total_w)):
    total_w = Q_cc[i]

hot_utilities = total_w - upper_bound


data_heat_recovery = {'Only with Rivella': ['Cold Utilities', 'Hot Utilities', 'Minimum Energy Requirements', 'Maximum Heat Recovery Potential'],
        'Heat (kW)': [cold_utilities, hot_utilities, cold_utilities+hot_utilities, np.round(upper_bound - lower_bound, 2)]}

dfs = pd.DataFrame(data_heat_recovery)
dfs = dfs.set_index('Only with Rivella')
dfs = dfs.round(2)
HTML(dfs.to_html())

In [ ]:
import matplotlib.pyplot as plt
import json
import numpy as np
import pandas as pd
from IPython.display import display, HTML

plt.switch_backend('agg')

# read json for heat integration data
file_name = open('codes_02_heat_recovery/result/brewery-59NoRivella.json')
integ = json.loads(file_name.read())

# we used .keys() and json-reader to locate the data of interest (hot and cold composite curves)
hotcc = integ["results"]["graph"][0][0][1]["data"][0]["curve"]
coldcc = integ["results"]["graph"][0][0][1]["data"][1]["curve"]

# extract the temperature (T in Kelvin) and heat (Q in kW)
T_hc = [i["T"] for i in hotcc]
Q_hc = [i["Q"] for i in hotcc]
T_cc = [i["T"] for i in coldcc]
Q_cc = [i["Q"] for i in coldcc]

# compute the shifted carnot factor (1 - T0/T) and Treference = 298.15K
Tref = 298.15 
ca_hc = [(1-Tref/i) for i in T_hc]
ca_cc = [(1-Tref/i) for i in T_cc]

# compute back the original carnot factors given a deltaT_min
#deltaT_min = 40
deltaT_min = 5
T_hc_ori = [(i+deltaT_min/2) for i in T_hc]
T_cc_ori = [(i-deltaT_min/2) for i in T_cc]
ca_hc_ori = [(1-298.15/i) for i in T_hc_ori]
ca_cc_ori = [(1-298.15/i) for i in T_cc_ori]

# prepare the second y-axis for plotting
# the minim carnot value is extracted from the original cooling. Round it to the lowest .0
# the maximum carnot carnot value is extracted from the original heating. Round it to the highest .0
ca_min = round(min(ca_cc_ori) - 0.05, 1)
ca_max = round(max(ca_hc_ori) + 0.05, 1)
# generate an array with the carnot list from min to max
ca_values = np.arange(ca_min, ca_max, 0.2)
# compute the corresponding temperature values in celcius
Tref_ce = 298.15
T_ce = [int(Tref_ce/(1-i) - 273.15) for i in ca_values]


T_cc = [(i-273.15) for i in T_cc]
T_cc_ori = [(i-273.15) for i in T_cc_ori]
T_hc = [(i-273.15) for i in T_hc]
T_hc_ori = [(i-273.15) for i in T_hc_ori]


Q_gcc = np.zeros(len(Q_cc))

for i in range(len(Q_cc)):
  Q_gcc[i] = Q_cc[i] - Q_hc[i]  

Heat_inter = []
intersections = []
for i in range(len(Q_gcc)):
  if (Q_gcc[i]<=10):
    if (T_hc[i]>=-30 and T_hc[i]<=200):
      intersections.append(T_hc[i])
      Heat_inter.append(Q_gcc[i])


upper_bound = 0
lower_bound = 30000
for i in range(len(Q_hc)):
  if (ca_hc_ori[i] < (np.max(ca_hc_ori)) and Q_hc[i] > upper_bound):
    upper_bound = Q_hc[i]
  if (ca_cc_ori[i] > np.min(ca_cc_ori) and Q_cc[i] < lower_bound):
    lower_bound = Q_cc[i]




cold_utilities = lower_bound
total_w = 0

for i in range(len(Q_cc)):
  if ((ca_cc_ori[i] > 0.2) and (Q_cc[i] > total_w)):
    total_w = Q_cc[i]

hot_utilities = total_w - upper_bound


data_heat_recovery = {'Only with Digester': ['Cold Utilities', 'Hot Utilities', 'Minimum Energy Requirements', 'Maximum Heat Recovery Potential'],
        'Heat (kW)': [cold_utilities, hot_utilities, cold_utilities+hot_utilities, np.round(upper_bound - lower_bound, 2)]}

dfs = pd.DataFrame(data_heat_recovery)
dfs = dfs.set_index('Only with Digester')
dfs = dfs.round(2)
HTML(dfs.to_html())

%1. Generating the Grand composite curve (GCC) of the process system and analysing the temperature levels and the amount for the heat deficits and surplus in the system. 

## System Key Performance Indicators (KPI)

To characterize the relevance of the system, Key Performance Indicators (KPI) are defined before being computed in the case of this study. The six following KPIs are used:


### KPIs definitions

|KPI Name|Acronym|Unit(s)| Description |
|:--|:--|:---|:----------------|
|Operating Expenditure|OPEX|[CHF/year]|Represents the cost of the resources and the waste flows to supply the energy requirements of the process on a yearly basis|
|Capital Expenditure|CAPEX|[CHF]|Indicates the investment associated with energy conversion units|
|Total Expenditure|TOTEX|[CHF/year]| Sums up the total cost of the energy supply considering the annualized investment and operating cost|
|Exergy Losses|-|[kW] or in [kJ/$kg_{product}$]| Evaluates the energy losses within the energy conversion technology implemented. Measured using [kJ/$kg_{product}$] when converting the resources to useful heat, electricity, cooling, and refrigeration needs|
|Emissions|-|[$kg_{CO_{2}eq}$/year] |Advises the emissions due to the energy conversion technology |      
|Life Cycle Environmental impact|-|[$kg_{CO_{2}eq}$/year]|Advises the emissions due to the life cycle of the energy conversion technology |      


### KPIs formulas

### Parameters

 @cinv


| Variable            | $cost_1$ | $cost_2$    | $cinv_1$       | $cinv_2$    | $imp_1$      | $imp_2$         | $f_{min}$ | $f_{max}$ |
|---------------------|-------|----------|-------------|----------|-----------|--------------|------|------|
| Unit                | [€/y] | [€/kW/y] | [€/y]       | [€/kW/y] | [kgCO2/y] | [kgCO2/kW/y] | %    | %    |
| Cooling Tower       | 0     | 10       | 0           | -        | 0         | 0            | 0    | 100  |
| Boiler              | 0     | 665,76   | 331,3013165 | -        | 0         | 230          | 0    | 100  |
| Cogeneration engine | 0     | 665,76   | 1355,022385 | -        | 0         | 230          | 0    | 100  |
| Heat Pump           | 0     | 499,32   | 495,2082836 | -        | 0         | 0            | 0    | 100  |
| Refrigeration       | 0     | 499,32   | 495,2082836 | -        | 0         | 0            | 0    | 100  |

## Adding Utilities to the Grand Composite Curve

### Dummy case

|        ICC Plot              |        Carnot Plot           |
| ---------------------------- | ----------------------------- |
| ![ICC Plot](Figures/icc_dummy.jpg) | ![Carnot Plot](Figures/carnot_dummy-1.jpg) |

### Optimization of case without Rivella and Digester

|        ICC Plot              |        Carnot Plot           |
| ---------------------------- | ----------------------------- |
| ![ICC Plot](Figures/icc_base.jpg) | ![Carnot Plot](Figures/carnot_base-1.jpg) |

### Optimization of case with Rivella

|        ICC Plot              |        Carnot Plot           |
| ---------------------------- | ----------------------------- |
| ![ICC Plot](Figures/icc_rivella.jpg) | ![Carnot Plot](Figures/carnot_rivella.jpg) |

### Optimization of case with Digester

|        ICC Plot              |        Carnot Plot           |
| ---------------------------- | ----------------------------- |
| ![ICC Plot](Figures/icc_digester.jpg) | ![Carnot Plot](Figures/carnot_digester.jpg) |

### Optimization of case with both Rivella and Digester

|        ICC Plot              |        Carnot Plot           |
| ---------------------------- | ----------------------------- |
| ![ICC Plot](Figures/icc_both.jpg) | ![Carnot Plot](Figures/carnot_both.jpg) |


You will proceed by the following steps :

1. Generating the Grand composite curve (GCC) of the process system and analysing the temperature levels and the amount for the heat deficits and surplus in the system. 
2. From the Grand composite curve analysis, a list of possible energy conversion technologies like boilers, steam or Rankine cycles, heat pumps, cooling water or refrigeration system will be established. The hot and cold streams as well as the resources consumption/production as well as the associated investment will be defined based on the estimated heat loads and temperatures from the process system GCC.
3. System **Key Performance Indicators (KPI)** will be defined to characterise :
     1. the operating expenditure (OPEX) in [CHF/year]: cost of the resources and the waste flows to supply the energy requirements of the process on a yearly basis.
     2. the capital expenditure (CAPEX) in [CHF]: the investment associated with energy conversion units
     3. the total expenditure (TOTEX) in [CHF/year]: total cost of the energy supply considering the annualised investment and operating cost
     3. the exergy losses in [kW] or in [kJ/$kg_{product}$] when converting hte resources to useful heat, electricity, cooling and refrigeration needs.
     4. the emissions  in [$kg_{CO_{2}}$/year] and the associated life cycle environmental impact in [$kg_{CO_{2}eq}$/year] of the energy supply.
4. The energy conversion system integration is calculated by calculating the relative flows (level of usage) in the energy conversion system. The flows are calculated based on a given set of energy conversion unit and with one of the KPI as an objective. This calculation aims at discovering synergies or complementarities between energy covnersion technologies to suppply the process energy needs.
5. In particular, the integration of the soda production will be investigated, by making a sensitivity analysis of the capacity on the soda production process.
6. The systematic generation of alternative system configuration options will allow to compare the different options and select the final recommendation.
4. The energy conversion system integration is calculated by calculating the relative flows (level of usage) in the energy conversion system. The flows are calculated based on a given set of energy conversion unit and with one of the KPI as an objective. This calculation aims at discovering synergies or complementarities between energy covnersion technologies to suppply the process energy needs.
5. In particular, the integration of the soda production will be investigated, by making a sensitivity analysis of the capacity on the soda production process.
6. The systematic generation of alternative system configuration options will allow to compare the different options and select the final recommendation.